In [1]:

!pip install diffusers transformers accelerate torch torchvision gradio pillow xformers -q
!pip install --upgrade gradio -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

In [13]:
# Advanced Stable Diffusion Image Generator - Fixed Version
# Compatible with all Gradio versions + Fast Generation + Activity History

import os
import torch
import gradio as gr
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from PIL import Image
import numpy as np
import time
import random
from datetime import datetime
import json
import threading
import gc

# ==================== STEP 1: INSTALL REQUIRED PACKAGES ====================
# Run this cell first in Colab:
"""
!pip install --upgrade pip
!pip install gradio==3.50.2
!pip install diffusers transformers accelerate torch torchvision pillow xformers -q
!pip install --upgrade huggingface_hub
"""

# Check Gradio version and use compatible syntax
print(f"Gradio version: {gr.__version__}")

class FastImageGenerator:
    def __init__(self):
        self.pipe = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model_id = "runwayml/stable-diffusion-v1-5"
        self.model_loaded = False
        self.generation_history = []
        self.activity_log = []

        print(f"🚀 Device: {self.device}")
        if self.device == "cuda":
            print(f"📊 GPU: {torch.cuda.get_device_name(0)}")
            print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

    def log_activity(self, action, details=""):
        """Log user activity"""
        activity = {
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "action": action,
            "details": details
        }
        self.activity_log.append(activity)
        if len(self.activity_log) > 50:  # Keep last 50 activities
            self.activity_log.pop(0)

    def load_model(self):
        """Load model with optimizations for speed"""
        try:
            self.log_activity("Model Loading", "Starting to load Stable Diffusion model")

            # Clear memory
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                gc.collect()

            print("📥 Loading Stable Diffusion model...")

            # Load with optimizations
            self.pipe = StableDiffusionPipeline.from_pretrained(
                self.model_id,
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
                safety_checker=None,
                requires_safety_checker=False,
                use_auth_token=False,
                low_cpu_mem_usage=True
            )

            # Fast scheduler for quicker generation
            self.pipe.scheduler = DPMSolverMultistepScheduler.from_config(
                self.pipe.scheduler.config,
                use_karras_sigmas=True
            )

            # Move to device
            self.pipe = self.pipe.to(self.device)

            # Enable all optimizations
            if self.device == "cuda":
                #self.pipe.enable_memory_efficient_attention()
                self.pipe.enable_attention_slicing()
                try:
                    self.pipe.enable_xformers_memory_efficient_attention()
                    print("✅ XFormers optimization enabled")
                except:
                    print("⚠️ XFormers not available, using standard attention")

            # Enable CPU offloading for large models
            self.pipe.enable_model_cpu_offload()

            self.model_loaded = True
            self.log_activity("Model Loaded", "Successfully loaded and optimized model")

            return "✅ Model loaded successfully! Ready for fast generation 🚀"

        except Exception as e:
            error_msg = f"❌ Error loading model: {str(e)}"
            self.log_activity("Model Load Error", str(e))
            print(f"Model loading error: {e}")
            return error_msg

    def generate_image(self, prompt, negative_prompt, width, height, steps, guidance_scale, seed, batch_size):
        """Generate images with optimized settings"""
        if not self.model_loaded:
            return None, "❌ Please load the model first!"

        try:
            self.log_activity("Image Generation", f"Prompt: {prompt[:50]}...")

            # Optimize parameters for speed
            if steps > 25:
                steps = 25  # Cap steps for faster generation

            # Set seed
            if seed == -1:
                seed = random.randint(0, 2**32 - 1)

            generator = torch.Generator(device=self.device).manual_seed(seed)

            print(f"🎨 Generating with optimized settings...")
            start_time = time.time()

            # Generate with optimizations
            with torch.autocast(self.device), torch.no_grad():
                result = self.pipe(
                    prompt=prompt,
                    negative_prompt=negative_prompt,
                    width=width,
                    height=height,
                    num_inference_steps=steps,
                    guidance_scale=guidance_scale,
                    generator=generator,
                    num_images_per_prompt=batch_size,
                    eta=0.0,  # Faster generation
                    output_type="pil"
                )

            end_time = time.time()
            generation_time = end_time - start_time

            # Store in history
            generation_record = {
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "prompt": prompt,
                "negative_prompt": negative_prompt,
                "seed": seed,
                "steps": steps,
                "guidance_scale": guidance_scale,
                "size": f"{width}x{height}",
                "time": f"{generation_time:.2f}s",
                "batch_size": batch_size
            }
            self.generation_history.append(generation_record)

            # Keep only last 20 generations
            if len(self.generation_history) > 20:
                self.generation_history.pop(0)

            self.log_activity("Image Generated", f"Generated {batch_size} images in {generation_time:.2f}s")

            images = result.images

            # Format info
            info_text = f"✅ Generated {len(images)} image(s) in {generation_time:.2f}s\n"
            info_text += f"🎲 Seed: {seed}\n"
            info_text += f"⚙️ Steps: {steps} | Guidance: {guidance_scale}\n"
            info_text += f"📐 Size: {width}x{height}\n"
            info_text += f"⚡ Speed optimizations: ON\n"
            info_text += f"💡 Tip: Lower steps = faster generation!"

            return images, info_text

        except Exception as e:
            error_msg = f"❌ Generation error: {str(e)}"
            self.log_activity("Generation Error", str(e))
            print(f"Generation error: {e}")
            return None, error_msg

    def get_random_prompt(self):
        """Generate random creative prompts"""
        styles = ["photorealistic", "digital art", "oil painting", "watercolor", "anime style", "cyberpunk", "fantasy art"]
        subjects = ["mountain landscape", "futuristic city", "magical forest", "space station", "ancient temple", "underwater scene"]
        moods = ["dramatic lighting", "soft glow", "vibrant colors", "golden hour", "neon lights", "mystical atmosphere"]
        quality = ["highly detailed", "8k", "masterpiece", "stunning", "cinematic"]

        return f"{random.choice(styles)} of {random.choice(subjects)} with {random.choice(moods)}, {random.choice(quality)}"

    def get_activity_history(self):
        """Get formatted activity history"""
        if not self.activity_log:
            return "No activities yet"

        history_text = "📋 Recent Activities:\n" + "="*50 + "\n"
        for activity in self.activity_log[-10:]:  # Show last 10 activities
            history_text += f"🕐 {activity['timestamp']}\n"
            history_text += f"🔸 {activity['action']}\n"
            if activity['details']:
                history_text += f"   {activity['details']}\n"
            history_text += "-" * 30 + "\n"

        return history_text

    def get_generation_history(self):
        """Get formatted generation history"""
        if not self.generation_history:
            return "No generations yet"

        history_text = "🎨 Generation History:\n" + "="*50 + "\n"
        for i, gen in enumerate(self.generation_history[-5:], 1):  # Show last 5 generations
            history_text += f"#{i} - {gen['timestamp']}\n"
            history_text += f"📝 Prompt: {gen['prompt'][:60]}...\n"
            history_text += f"🎲 Seed: {gen['seed']} | ⚙️ Steps: {gen['steps']}\n"
            history_text += f"📐 Size: {gen['size']} | ⏱️ Time: {gen['time']}\n"
            history_text += "-" * 40 + "\n"

        return history_text

# Initialize generator
generator = FastImageGenerator()

# Enhanced CSS for better performance
custom_css = """
body {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
}

.gradio-container {
    background: transparent !important;
}

.gr-button {
    border-radius: 20px !important;
    font-weight: bold !important;
    transition: all 0.2s ease !important;
}

.gr-button.primary {
    background: linear-gradient(45deg, #4CAF50, #45a049) !important;
    color: white !important;
    border: none !important;
}

.gr-button.secondary {
    background: linear-gradient(45deg, #2196F3, #1976D2) !important;
    color: white !important;
    border: none !important;
}

.generate-button {
    background: linear-gradient(45deg, #FF5722, #F44336) !important;
    color: white !important;
    font-size: 18px !important;
    padding: 15px 30px !important;
    border-radius: 25px !important;
    font-weight: bold !important;
}

.header {
    text-align: center;
    color: white;
    padding: 20px;
    background: rgba(255,255,255,0.1);
    border-radius: 15px;
    margin-bottom: 20px;
    backdrop-filter: blur(10px);
}

.status-box {
    background: rgba(255,255,255,0.1) !important;
    color: white !important;
    border-radius: 10px !important;
    padding: 10px !important;
    font-weight: bold !important;
}

.info-display {
    background: rgba(255,255,255,0.1) !important;
    color: white !important;
    border-radius: 10px !important;
    padding: 15px !important;
    font-family: monospace !important;
    font-size: 14px !important;
    border-left: 4px solid #4CAF50 !important;
}

.history-display {
    background: rgba(255,255,255,0.1) !important;
    color: white !important;
    border-radius: 10px !important;
    padding: 15px !important;
    font-family: monospace !important;
    font-size: 12px !important;
    border-left: 4px solid #2196F3 !important;
    max-height: 300px !important;
    overflow-y: auto !important;
}
"""

# Preset prompts for quick start
preset_prompts = [
    "A majestic dragon soaring over mountains, fantasy digital art, highly detailed",
    "Cyberpunk cityscape at night, neon lights, futuristic, 8k resolution",
    "Beautiful cherry blossom tree in spring, soft lighting, photorealistic",
    "Steampunk robot in Victorian setting, brass and copper, intricate details",
    "Magical underwater city with glowing coral, fantasy art, vibrant colors",
    "Space explorer on alien planet, sci-fi, dramatic lighting, cinematic",
    "Ancient mystical forest with fairy lights, ethereal atmosphere, enchanting",
    "Modern architectural marvel, glass and steel, sunset lighting, photography"
]

def create_fast_interface():
    """Create optimized interface compatible with all Gradio versions"""

    # Create interface using compatible syntax
    with gr.Blocks(css=custom_css, title="Fast AI Image Generator") as app:

        # Header
        gr.HTML("""
        <div class="header">
            <h1>🚀 Fast AI Image Generator</h1>
            <p>⚡ Optimized for Speed | 🎨 Powered by Stable Diffusion</p>
            <p>🌐 <strong>Your public URL will appear in the console output below</strong></p>
        </div>
        """)

        with gr.Row():
            with gr.Column(scale=1):
                # Model Section
                gr.Markdown("### 🤖 Model Control")
                load_model_btn = gr.Button("🚀 Load Fast Model", variant="primary")
                model_status = gr.Textbox(
                    label="Model Status",
                    value="🔴 Click 'Load Fast Model' to start",
                    interactive=False
                )

                # Quick Settings
                gr.Markdown("### ⚙️ Quick Settings")
                with gr.Row():
                    width = gr.Slider(256, 1024, 512, step=64, label="Width")
                    height = gr.Slider(256, 1024, 512, step=64, label="Height")

                with gr.Row():
                    steps = gr.Slider(5, 25, 15, step=1, label="Steps (lower = faster)")
                    guidance = gr.Slider(3, 15, 7, step=0.5, label="Guidance Scale")

                with gr.Row():
                    seed = gr.Number(-1, label="Seed (-1 = random)")
                    batch_count = gr.Slider(1, 4, 1, step=1, label="Images Count")

                # Quick Actions
                gr.Markdown("### 🎯 Quick Actions")
                preset_dropdown = gr.Dropdown(
                    choices=preset_prompts,
                    label="Example Prompts",
                    value=preset_prompts[0]
                )

                with gr.Row():
                    use_preset_btn = gr.Button("Use Example", variant="secondary")
                    random_btn = gr.Button("Random Prompt", variant="secondary")

                # Activity History
                gr.Markdown("### 📊 My Activity")
                activity_btn = gr.Button("Show Activity", variant="secondary")
                activity_display = gr.Textbox(
                    label="Activity History",
                    lines=8,
                    interactive=False,
                    visible=False
                )

                # Generation History
                history_btn = gr.Button("Show Generation History", variant="secondary")
                history_display = gr.Textbox(
                    label="Generation History",
                    lines=8,
                    interactive=False,
                    visible=False
                )

            with gr.Column(scale=2):
                # Prompts
                gr.Markdown("### 💭 Describe Your Image")
                prompt_input = gr.Textbox(
                    label="Main Prompt",
                    placeholder="A beautiful landscape...",
                    lines=3,
                    value="A beautiful sunset over mountains with a lake, digital art, highly detailed"
                )

                negative_input = gr.Textbox(
                    label="Negative Prompt (what to avoid)",
                    placeholder="blurry, low quality...",
                    lines=2,
                    value="blurry, low quality, distorted, ugly, bad anatomy"
                )

                # Generate Button
                generate_btn = gr.Button("🎨 Generate Images Fast!", variant="primary", elem_classes="generate-button")

                # Status Display
                generation_info = gr.Textbox(
                    label="Generation Status",
                    lines=8,
                    interactive=False,
                    value="🎯 Ready! Load model and click generate to start creating amazing images!"
                )

        # Gallery
        gr.Markdown("### 🖼️ Generated Images")
        image_gallery = gr.Gallery(
            label="Your Images",
            show_label=False,
            elem_id="gallery",
            columns=2,
            rows=2
        )

        # Event Handlers
        load_model_btn.click(
            generator.load_model,
            outputs=[model_status]
        )

        generate_btn.click(
            generator.generate_image,
            inputs=[prompt_input, negative_input, width, height, steps, guidance, seed, batch_count],
            outputs=[image_gallery, generation_info]
        )

        use_preset_btn.click(
            lambda x: x,
            inputs=[preset_dropdown],
            outputs=[prompt_input]
        )

        random_btn.click(
            generator.get_random_prompt,
            outputs=[prompt_input]
        )

        activity_btn.click(
            lambda: (generator.get_activity_history(), True),
            outputs=[activity_display, activity_display]
        )

        history_btn.click(
            lambda: (generator.get_generation_history(), True),
            outputs=[history_display, history_display]
        )

    return app

def launch_app():
    """Launch the application with URL display"""
    print("=" * 60)
    print("🚀 LAUNCHING FAST AI IMAGE GENERATOR")
    print("=" * 60)

    # Create and launch
    app = create_fast_interface()

    print("\n📡 Starting server with public URL...")

    # Launch with public sharing (Fixed: Removed invalid 'show_tips' argument)
    app.launch(
        share=True,
        server_name="0.0.0.0",
        server_port=7864,
        inbrowser=True,
    )



# Run the application
if __name__ == "__main__":
    launch_app()

Gradio version: 5.31.0
🚀 Device: cuda
📊 GPU: Tesla T4
💾 VRAM: 14.7 GB
🚀 LAUNCHING FAST AI IMAGE GENERATOR

📡 Starting server with public URL...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4af3f88a720b205bf6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
